<img src="https://i.imgur.com/6U6q5jQ.png"/>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/TableOperations/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Operations on Data Frames


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [1]:
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/TableOperations/raw/main/dengue_ok.pkl"

dengue = pd.read_pickle(linkData)

# checking format
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501236 entries, 0 to 501235
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   departamento  501236 non-null  object  
 1   provincia     501236 non-null  object  
 2   distrito      501236 non-null  object  
 3   ano           501236 non-null  int64   
 4   semana        501236 non-null  int64   
 5   sexo          501236 non-null  object  
 6   edad          501236 non-null  int64   
 7   enfermedad    501236 non-null  category
 8   case          501236 non-null  int64   
dtypes: category(1), int64(4), object(4)
memory usage: 31.1+ MB


In [2]:
# Each row is a person:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,1


In [3]:
# some exploration
dengue.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,ano,semana,edad,case
count,501236.00000,501236.00000,501236.00000,501236.00000
mean,2014.77213,21.99838,28.96143,1.00000
std,6.14646,14.76658,18.15954,0.00000
min,2000.00000,1.00000,0.00000,1.00000
25%,2011.00000,11.00000,15.00000,1.00000
50%,2016.00000,18.00000,26.00000,1.00000
75%,2020.00000,32.00000,41.00000,1.00000
max,2022.00000,53.00000,106.00000,1.00000


In [4]:
# exploring
dengue.enfermedad.value_counts()

,count
enfermedad,
SIN_SEÑALES,443996
ALARMA,54981
GRAVE,2259


Better labels:

In [5]:
dengue['enfermedad_text']=dengue.enfermedad.astype(str)

dengue.replace({'enfermedad_text':{'SIN_SEÑALES':'1_SIN_SEÑALES','ALARMA':'2_ALARMA','GRAVE':'3_GRAVE'}},inplace=True)

In [6]:
# exploring
dengue.ano.value_counts(sort=False)

,count
ano,
2000,5557
2001,23526
2002,8086
2003,3349
2004,9547
2005,5640
2006,4022
2007,6344
2008,12824


Discretizing:

In [7]:
binLimits=[0,15,50,110]
theLabels=["a_menor_a_16","b_entre_16y50","c_mayor_a_50"]
dengue["edad_grupos"]=pd.cut(dengue['edad'], include_lowest=True,
                                     bins=binLimits,
                                     labels=theLabels,
                                     ordered=True)

# see

dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,case,enfermedad_text,edad_grupos
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,1,1_SIN_SEÑALES,a_menor_a_16
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,1,1_SIN_SEÑALES,b_entre_16y50
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,1,1_SIN_SEÑALES,b_entre_16y50
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,1,1_SIN_SEÑALES,b_entre_16y50
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,1,1_SIN_SEÑALES,b_entre_16y50


The surface:

In [8]:
pd.crosstab( dengue.enfermedad_text,dengue.edad_grupos, dropna=False, normalize='columns')

edad_grupos,a_menor_a_16,b_entre_16y50,c_mayor_a_50
enfermedad_text,,,
1_SIN_SEÑALES,0.876868,0.890247,0.884549
2_ALARMA,0.119010,0.105475,0.109204
3_GRAVE,0.004122,0.004278,0.006248


In [9]:
pd.crosstab(dengue.enfermedad_text,[dengue.sexo,dengue.edad_grupos], dropna=False, normalize='columns')

sexo                       F                                       M  \
edad_grupos     a_menor_a_16 b_entre_16y50 c_mayor_a_50 a_menor_a_16   
enfermedad_text                                                        
1_SIN_SEÑALES       0.875221      0.884646     0.881328     0.878431   
2_ALARMA            0.120614      0.110622     0.113616     0.117488   
3_GRAVE             0.004165      0.004731     0.005057     0.004081   

sexo                                        
edad_grupos     b_entre_16y50 c_mayor_a_50  
enfermedad_text                             
1_SIN_SEÑALES        0.897139     0.888295  
2_ALARMA             0.099142     0.104073  
3_GRAVE              0.003720     0.007633

# Yearly look

In [10]:
# for colab
# !pip install altair -U
# # !pip install "vegafusion-jupyter[embed]"

In [11]:
import altair as alt
alt.data_transformers.enable("vegafusion")


DataTransformerRegistry.enable('vegafusion')

In [12]:
alt_dengue=alt.Chart(dengue)

enc_dengue=alt_dengue.encode(
    x='ano:T',
    y='mean(edad):Q',
    color='enfermedad_text:N',
)

enc_dengue.mark_line() + enc_dengue.mark_errorband()

More detailed:

In [13]:
enc_dengue=alt_dengue.encode(
    x='ano:T',
    y='median(edad):Q',
    color='enfermedad_text:N',
    tooltip=['median(edad)','ano:T']
).interactive()

enc_dengue.mark_line().facet(
    row='sexo:N',
    column='edad_grupos:N'
)

alt.FacetChart(...)

In [ ]:
enc_dengue=alt_dengue.encode(
    x='ano:T',
    y=alt.Y('sum(case):Q'),
    color='enfermedad_text:N',
    tooltip=['sum(case):Q','ano:T']
).interactive()
enc_dengue.mark_line().facet(
    row='sexo:N',
    column='edad_grupos:N'
)

The previous plot may require a logged Y-axis:

In [ ]:
enc_dengue=alt_dengue.encode(
    x='ano:T',
    y=alt.Y('sum(case):Q', scale=alt.Scale(type='log')),
    color='enfermedad_text:N',
    tooltip=['sum(case):Q','ano:T']
).interactive()

enc_dengue.mark_line().facet(
    row='sexo:N',
    column='edad_grupos:N'
)

Let's get the same results in tables:

In [ ]:
indexList=['edad_grupos','ano','sexo','enfermedad_text']
aggregator={'edad': ['median']}
LevelByYear_medians=dengue.groupby(indexList,observed=True).agg(aggregator)
LevelByYear_medians

In [ ]:
LevelByYear_medians.unstack(['sexo','enfermedad_text'])

Notice the multi-index:

In [ ]:
LevelByYear_medians.info()

These are other possibilities, but not better than the lines:

In [ ]:
alt_dengue=alt.Chart(dengue)
enc_dengue=alt_dengue.encode(
    x='ano:T',
    y=alt.Y('sum(case):Q', scale=alt.Scale(type='log')),
    column='enfermedad_text:N'
)
enc_dengue.mark_circle()

In [ ]:
alt_dengue=alt.Chart(dengue)
enc_dengue=alt_dengue.encode(
    x='ano:T',
    y=alt.Y('sum(case):Q', scale=alt.Scale(type='log')),
    column='enfermedad_text:N',
)
enc_dengue.mark_rule()

In [ ]:
alt_dengue=alt.Chart(dengue)
enc_dengue=alt_dengue.encode(
    x='ano:T',
    y=alt.Y('sum(case):Q', scale=alt.Scale(type='log')),
    column='enfermedad_text:N',
)
enc_dengue.mark_bar()

Let's do some aggregation:

In [ ]:
indexList=['edad_grupos','ano','sexo','enfermedad_text']
aggregator={'edad': ['median','mean','min','max']}
LevelByYear_statsFull=dengue.groupby(indexList,observed=True).agg(aggregator)
LevelByYear_statsFull

Now, some reshaping:

In [ ]:
LevelByYear_statsFull.stack(future_stack=True)

# Mining location

Let's use _departamento_ and _provincia_:

In [ ]:
indexList=['ano','departamento','provincia','enfermedad_text']
aggregator={'case':['sum']}
ByYearPlace=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace

Create a wide shape:

In [ ]:
#long to wide
ByYearPlace.unstack()

In [ ]:
# no missing values
ByYearPlace_wide=ByYearPlace.unstack().fillna(0)
ByYearPlace_wide

The idea is get the sgare of people in ALARM status. For that we need this:

In [ ]:
sumCases=ByYearPlace_wide.sum(axis=1)
sumCases

In [ ]:
# here you are:
shareAlarma=ByYearPlace_wide.loc[:,('case','sum','2_ALARMA')]/sumCases
shareAlarma.name='shareAlarma'
shareAlarma

No multi index:

In [ ]:
shareAlarma=shareAlarma.reset_index()
shareAlarma

Let's find thwe worst province per Region in a year:

In [ ]:
where = shareAlarma.groupby(['ano','departamento'])['shareAlarma'].idxmax()
worst_prov_year = shareAlarma.loc[where].reset_index(drop=True)
worst_prov_year

In [ ]:
worst_prov_year.shareAlarma.describe()

In [ ]:
# amount of worst provinces per region
len(worst_prov_year.provincia.value_counts())

In [ ]:
# amount of worst provinces per region - cleaner
len(worst_prov_year[worst_prov_year.shareAlarma>0].provincia.value_counts())

Some filtering:

In [ ]:
worst_ProvYear_alarma=worst_prov_year[worst_prov_year.shareAlarma>0].loc[:,['departamento','provincia']]
worst_ProvYear_alarma.reset_index(drop=True,inplace=True)
worst_ProvYear_alarma

In [ ]:
indexList=['departamento','provincia']
aggregator={'provincia':['count']}
worst_ProvYear_alarma_Frequency=worst_ProvYear_alarma.groupby(indexList,observed=True).agg(aggregator)
worst_ProvYear_alarma_Frequency

The count informs how many years a province was the most affected:

In [ ]:
worst_ProvYear_alarma_Frequency.describe()

In [ ]:
# final look
worst_ProvYear_alarma_Frequency.columns=['yearsAffected']
worst_ProvYear_alarma_Frequency=worst_ProvYear_alarma_Frequency[worst_ProvYear_alarma_Frequency.yearsAffected>2]
worst_ProvYear_alarma_Frequency.reset_index(inplace=True)
worst_ProvYear_alarma_Frequency

Let's plot:

In [ ]:
alt_worstProv=alt.Chart(worst_ProvYear_alarma_Frequency)

enc_worstProv=alt_worstProv.encode(
    y='departamento',
    x='provincia',
    text='yearsAffected:O',
    size='yearsAffected:O'
)

enc_worstProv.mark_text()

Let's try another info:

In [ ]:
indexList=['ano','departamento','enfermedad_text']
aggregator={'case':['sum']}
ByYearDepa=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYearDepa_wide=ByYearDepa.unstack().fillna(0)
ByYearDepaAlarm=ByYearDepa_wide.loc[:,('case','sum','2_ALARMA')]/ByYearDepa_wide.sum(axis=1)
ByYearDepaAlarm.name='alarmShare'

ByYearDepaAlarm=ByYearDepaAlarm.reset_index()
ByYearDepaAlarm

In [ ]:
ByYearDepaAlarm.describe()

In [ ]:
ByYearDepaAlarm_focus=ByYearDepaAlarm[ByYearDepaAlarm.alarmShare>0]

In [ ]:
ByYearDepaAlarm_focus.describe()

In [ ]:
edges=[-1, .10, .25, .5,1]
theLabels=["a.below10%","b.11-25%","c.26-50%","d.above50%"]
ByYearDepaAlarm_focus.loc[:,"alarmLevels"]=pd.cut(ByYearDepaAlarm_focus['alarmShare'],
                                            include_lowest=True,
                                            bins=edges,
                                            labels=theLabels,
                                            ordered=True)

##
ByYearDepaAlarm_focus.head()

In [ ]:
alt_WorstDepa=alt.Chart(ByYearDepaAlarm_focus).encode(x='ano:O',
                                                      y=alt.Y('departamento:N',
                                                              sort=alt.EncodingSortField(field='alarmShare',op='max',order='descending')))
enc1_WorstDepa=alt_WorstDepa.encode(
    color=alt.Color('alarmLevels:O').scale(scheme="lightgreyred", reverse=False)
)

enc1_WorstDepa.mark_rect()

In [ ]:
enc2_WorstDepa=alt_WorstDepa.encode(
    text=alt.Text('alarmShare:Q', format=".1f"),
    opacity=alt.condition('datum.alarmShare >= 0.3', alt.value(1), alt.value(0)))
enc2_WorstDepa.mark_text(fontStyle='bold')

In [ ]:
enc1_WorstDepa.mark_rect() + enc2_WorstDepa.mark_text()

You can find different color schemes [here](https://vega.github.io/vega/docs/schemes/)